In [1]:
import os
import re
import numpy as np
import pandas as pd
import json
from pprint import pprint
from tqdm import tqdm_notebook as tqdm
from multiprocessing import Pool, cpu_count

### Determining food names and tables

In [2]:
path = "../raw/www.bda-ieo.it"
foods = [food for food in os.listdir(path) if food[0] != "."]

### Merging csv files

In [3]:
def create_row(food):
    path = "../raw/www.bda-ieo.it"
    file_path = "{path}/{name}/0.csv".format(path=path, name=food)
    df = pd.read_csv(file_path, usecols=("0", "1"))
    for index, value in df.iterrows():
        if pd.isna(value).any():
            df = df.drop(index)
    df= df.reset_index().drop(columns=["index"])
    df.columns = df.iloc[0]
    df.index = df["Componenti Alimentari"]
    df = df.drop(columns=["Componenti Alimentari"])
    df = df.drop("Componenti Alimentari")
    df = df.drop([c for c in df.index if "kJ" in c])
    for index, value in df.iterrows():
        value = value["Valore"]
        if isinstance(value, str) and "tr" in value:
            value = 0.05
        if any([unit in index for unit in ["mg", "mcg"]]):
            if ", mg" in index:
                value = float(value)*1e-3
            elif ", mcg" in index or ", µg" in index:
                value = float(value)*1e-6
        else:
            value = float(value)
        df.loc[index] = value
        replace = {
            "mg":"g",
            "µg": "g",
            "mcg": "g",
            "kcal":"kcal",
            "g":"g",
            "%": "%"
        }
        new_index = index
        for unit in replace:
            new_index = new_index.replace(", {unit}".format(unit=unit), " | {unit}".format(unit=replace[unit]))
        new_index = new_index.replace("  ", " ")
        df = df.rename(index={index:new_index})
    df = df.transpose()
    del df.columns.name
    df.index = [food]
    df.index.name = "name"
    return df.astype(float)

In [4]:
from IPython.display import display

with Pool(cpu_count()) as p:
    bda = pd.concat(list(tqdm(p.imap(create_row, foods), total=len(foods))))
bda.to_csv("../csv/bda.csv")